In [42]:
import pandas as pd
import numpy as np
import math
import json
import os

from KDEpy import FFTKDE
from scipy import stats
from scipy.signal import argrelextrema, argrelmin

from tqdm import tqdm

from matplotlib.pyplot import figure
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
sns.set(rc={'figure.figsize':(24, 15)}, font_scale=2)

from glob import glob

import sys
sys.path.append('../')
import utils

# Functions

In [43]:
# KDE Bandwidth using ISJ Functions

def get_outlier_timestamps(df):
    '''
    Given a dataframe `df` of the processed time series return as a numpy array the timestamps of all detected outliers

    If a timestamps corresponds to an outlier from multiple views then the timestamp is repeated for each view 
    (i.e, there can be repetitions of the same timestamp)
    '''
    # Get the timestamps of all outliers
    outliers_df = df.copy()[df['raw_voting_score'] > 0]
    points = []
    
    # Each vote adds one point (a timestamp with more than 1 votes is added more than once)
    for _, row in outliers_df.iterrows():
        points += [row["unix_time"]] * row['raw_voting_score']

    return np.array(points)

def get_local_minima(arr):
    '''
    Given a list `arr` find its local minima.
    If there are flat minima present then only pick the boundaries of the minima flat regions
    '''
    minima_all = argrelextrema(arr, np.less_equal)[0]

    # Refine the minima list to avoid flat minima
    minima = []
    for min_idx in minima_all:
        # Loop over indices that aren't at the boundaries of the input list
        if ((min_idx > 0) and (min_idx < (len(arr) - 1))):
            if arr[min_idx -1] != arr[min_idx + 1]:
                minima.append(min_idx)
    
    return minima

def get_kde_isj_density_df(df):
    '''
    Run KDE over the timeseries in `df` using Improved Sheather-Jones (ISJ) bandwidth selection

    Returns a dataframe with the density
    '''
    # Get a list of the outlier timestamps
    outlier_timestamps = get_outlier_timestamps(df)

    # Sample the timestamps for which we estimate the density (use 3X num points of the df)
    samples = np.linspace(df['unix_time'].min()-3600,df['unix_time'].max()+3600, num=3*len(df.index))

    # Co                                                                                                                                                                                                                                                                                                                                                                    mpute density estimates using 'ISJ' - Improved Sheather Jones
    density = FFTKDE(kernel='epa', bw='ISJ').fit(outlier_timestamps).evaluate(samples)

    # Create the density_df
    density_df = pd.DataFrame({'unix_time': samples, 'density': density})
    density_df['timestamp'] = pd.to_datetime(density_df['unix_time'], unit='s')
    return density_df

def get_isj_clusters_df(density_df, df):
    mi = get_local_minima(density_df['density'].values)
    clusters_df_ISJ = utils.clustering.get_clusters_df(
        samples=density_df['unix_time'].values, density_dist=density_df['density'].values, minima_idx_list=mi,
        points=get_outlier_timestamps(df)
    )
    clusters_df_ISJ = clusters_df_ISJ.sort_values(by='area', ascending=False)
    return clusters_df_ISJ

def get_timeseries_and_density_figure(df, density, clusters, k):
    df = df.sort_values(by='timestamp')

    k_clusters = clusters.sort_values(by='area', ascending=False).head(k).reset_index()
    k_clusters

    # Plot the Figure
    fig, axs = plt.subplots(2, sharex=True)
    fig.subplots_adjust(hspace=0)

    axs[0].plot(df['unix_time'], df['measure'])
    axs[0].scatter(df[df['is_outlier']==1]['unix_time'].values, df[df['is_outlier']==1]['measure'].values, s=90, c='red', zorder=10, label='True Injected Outliers')
    axs[0].scatter(df[df['raw_voting_score']>0]['unix_time'].values, df[df['raw_voting_score']>0]['measure'].values, s=30, c='green', zorder=10, label='Detected Outliers')
    axs[0].set_ylabel('Measure');axs[0].legend()

    # Plot the alert regions
    for _, alert in k_clusters.iterrows():
        axs[0].add_patch(patches.Rectangle((alert['start'], -1.5), width=alert['end']-alert['start'], height=3, linewidth=0, color='yellow', zorder=10, alpha=0.40))

    axs[1].plot(density['unix_time'], density['density'])
    axs[1].set_ylabel('Density');axs[1].set_xlabel('Unix Time')
    for idx, alert in k_clusters.iterrows():
        axs[1].add_patch(patches.Rectangle((alert['start'], 0), width=alert['end']-alert['start'], height=density['density'].max(), linewidth=0, color='yellow', zorder=10, alpha=0.40))
        x_loc = alert['start'] + (alert['end'] - alert['start'])/2
        axs[1].text(x=x_loc, y=density['density'].max(), s=str(idx+1), fontsize=12, color='red', horizontalalignment='center')

    plt.tight_layout()
    plt.close()

    return fig

In [44]:
# Evaluation Related Functions

def get_real_and_predicted_ranges(regions_df, clusters_df, cluster_points_bounds=False):
    '''
    Given the `regions_df` and the `clusters_df` extract the real and predicted anomalous regions
    
    Notice that the predicted anomalous regions are set to be equal to the number of the true regions
    and the order they are picked is specified by the order of the rows in the `clusters_df`
    '''
    # Extract the real anomaly ranges and the predicted anomaly ranges
    real_ranges = []
    predicted_ranges = []
    for _, row in regions_df.iterrows():
        real_ranges.append([row['unix_start'], row['unix_end']])
    for _, row in clusters_df.iterrows():
        if cluster_points_bounds:
            predicted_ranges.append([row['point_start'], row['point_end']])
        else:
            predicted_ranges.append([row['start'], row['end']])
    predicted_ranges = predicted_ranges[:len(real_ranges)]

    return real_ranges, predicted_ranges

In [45]:
def get_eval_df_over_dir(dir_paths, cluster_points_bounds=False):
    precision_list = [];recall_list = []
    precision_isj_list =[];recall_isj_list = []
    precision_isj_tight_list=[];recall_isj_tight_list=[]

    for path in tqdm(dir_paths):
        regions_df = pd.read_pickle(path + 'regions_df.pickle')
        
        df = pd.read_pickle(path + 'df.pickle').sort_values(by='unix_time')
        clusters_df = pd.read_pickle(path + 'clusters_df.pickle').sort_values(by='area', ascending=False)

        # Add unix time to the regions_df
        regions_df['unix_start'] = df.loc[regions_df['start']]['unix_time'].values
        regions_df['unix_end'] = df.loc[regions_df['end']]['unix_time'].values

        # Compute the range-based precision and recall
        real_ranges, predicted_ranges = get_real_and_predicted_ranges(regions_df, clusters_df, cluster_points_bounds=cluster_points_bounds)
        recall = utils.metrics.range_based_recall(real_ranges=real_ranges, predicted_ranges=predicted_ranges, alpha=1, delta_mode='flat')
        precision = utils.metrics.range_based_precision(real_ranges=real_ranges, predicted_ranges=predicted_ranges, delta_mode='flat')
        precision_list.append(precision);recall_list.append(recall)

        # Get clusters using bandwidth from ISJ 
        density_df_ISJ = get_kde_isj_density_df(df)
        clusters_df_ISJ = get_isj_clusters_df(density_df_ISJ, df)

        # Save the density_df and cluster_df for ISJ in the directory
        density_df_ISJ.to_pickle(path+'density_ISJ_df.pickle')
        clusters_df_ISJ.to_pickle(path+'clusters_ISJ_df.pickle')

        # Save the timeseries_density plot using ISJ bandwidth
        fig = get_timeseries_and_density_figure(df, density_df_ISJ, clusters_df_ISJ, k=len(regions_df.index))
        fig.savefig(path+'figures/timeseries_density_ISJ_plot.svg')  

        # Compute the range-based precision and recall with ISJ bandwidth
        real_ranges, predicted_ranges = get_real_and_predicted_ranges(regions_df, clusters_df_ISJ, cluster_points_bounds=True)
        recall_isj = utils.metrics.range_based_recall(real_ranges=real_ranges, predicted_ranges=predicted_ranges, alpha=1, delta_mode='flat')
        precision_isj = utils.metrics.range_based_precision(real_ranges=real_ranges, predicted_ranges=predicted_ranges, delta_mode='flat')
        precision_isj_list.append(precision_isj);recall_isj_list.append(recall_isj)

        # Compute range-based precision and recall with ISJ bandwidth but tight
        real_ranges, predicted_ranges = get_real_and_predicted_ranges(regions_df, clusters_df_ISJ, cluster_points_bounds=cluster_points_bounds)
        recall_tight_isj = utils.metrics.range_based_recall(real_ranges=real_ranges, predicted_ranges=predicted_ranges, alpha=1, delta_mode='flat')
        precision_tight_isj = utils.metrics.range_based_precision(real_ranges=real_ranges, predicted_ranges=predicted_ranges, delta_mode='flat')
        precision_isj_tight_list.append(precision_tight_isj);recall_isj_tight_list.append(recall_tight_isj)
    
    evaluation_dict = {
        'precision': precision_list, 'recall': recall_list, 'precision_isj': precision_isj_list, 'recall_isj': recall_isj_list, 'precision_isj_tight':  precision_isj_tight_list, 'recall_isj_tight': recall_isj_tight_list,
        'seed': list(range(1, len(precision_list) + 1))
    }
    evaluation_df = pd.DataFrame.from_dict(evaluation_dict)
    evaluation_df['f1_score'] = (2 * evaluation_df['precision'].to_numpy() * evaluation_df['recall'].to_numpy()) / (evaluation_df['precision'].to_numpy() + evaluation_df['recall'].to_numpy())
    evaluation_df['f1_score_isj'] = (2 * evaluation_df['precision_isj'].to_numpy() * evaluation_df['recall_isj'].to_numpy()) / (evaluation_df['precision_isj'].to_numpy() + evaluation_df['recall_isj'].to_numpy())
    evaluation_df['f1_score_isj_tight'] = (2 * evaluation_df['precision_isj_tight'].to_numpy() * evaluation_df['precision_isj_tight'].to_numpy()) / (evaluation_df['precision_isj_tight'].to_numpy() + evaluation_df['precision_isj_tight'].to_numpy())

    return evaluation_df

# Meanshift5 Experiment

In [48]:
dir_paths = sorted(glob("../synthetic_data/mdi_synthetic_data/meanshift5/*/"))
evaluation_df = get_eval_df_over_dir(dir_paths)

print("ISJ Evaluation:")
print("Mean/SDEV: Range-Based Precision:", evaluation_df['precision_isj'].mean(), evaluation_df['precision_isj'].std())
print("Mean/SDEV Range-Based Recall:", evaluation_df['recall_isj'].mean(), evaluation_df['recall_isj'].std())
print("Mean/SDEV F1-Score:", evaluation_df['f1_score_isj'].mean(), evaluation_df['f1_score_isj'].std())

100%|██████████| 100/100 [00:21<00:00,  4.56it/s]

ISJ Evaluation:
Mean/SDEV: Range-Based Precision: 0.7875461349935235 0.12432242062212612
Mean/SDEV Range-Based Recall: 0.6274999999999998 0.12640623146639315
Mean/SDEV F1-Score: 0.6901761827115953 0.10377302098730616


In [49]:
evaluation_df.mean()

precision               0.175090
recall                  0.965500
precision_isj           0.787546
recall_isj              0.627500
precision_isj_tight     0.560800
recall_isj_tight        0.639500
seed                   50.500000
f1_score                0.294958
f1_score_isj            0.690176
f1_score_isj_tight      0.560800
dtype: float64

# Meanshift5_hard

In [50]:
dir_paths = sorted(glob("../synthetic_data/mdi_synthetic_data/meanshift5_hard/*/"))
evaluation_df = get_eval_df_over_dir(dir_paths)

print("ISJ Evaluation:")
print("Mean/SDEV: Range-Based Precision:", evaluation_df['precision_isj'].mean(), evaluation_df['precision_isj'].std())
print("Mean/SDEV Range-Based Recall:", evaluation_df['recall_isj'].mean(), evaluation_df['recall_isj'].std())
print("Mean/SDEV F1-Score:", evaluation_df['f1_score_isj'].mean(), evaluation_df['f1_score_isj'].std())

100%|██████████| 99/99 [00:22<00:00,  4.47it/s]

ISJ Evaluation:
Mean/SDEV: Range-Based Precision: 0.5283452644553105 0.23852645946288176
Mean/SDEV Range-Based Recall: 0.4777777777777778 0.17514166601526654
Mean/SDEV F1-Score: 0.49813067822704593 0.18229170655895502



<ipython-input-45-6dc4e2729d19>:52: RuntimeWarning: invalid value encountered in true_divide
  evaluation_df['f1_score_isj'] = (2 * evaluation_df['precision_isj'].to_numpy() * evaluation_df['recall_isj'].to_numpy()) / (evaluation_df['precision_isj'].to_numpy() + evaluation_df['recall_isj'].to_numpy())
<ipython-input-45-6dc4e2729d19>:53: RuntimeWarning: invalid value encountered in true_divide
  evaluation_df['f1_score_isj_tight'] = (2 * evaluation_df['precision_isj_tight'].to_numpy() * evaluation_df['precision_isj_tight'].to_numpy()) / (evaluation_df['precision_isj_tight'].to_numpy() + evaluation_df['precision_isj_tight'].to_numpy())
